In [17]:
#r ".\binaries\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 0
Count: 0



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\Users\jakob\Documents\Uni\Promotion\Programmieren\BoSSS\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 217
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\Users\jakob\Documents\Uni\Promotion\Programmieren\BoSSS\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 104
   at Submission#18.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [18]:
using BoSSS.Solution.Tecplot;
using ilPSP.Connectors.Matlab;
using static ilPSP.IMatrixExtensions;
using System.IO;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.CompressibleFlowCommon.ShockFinding;
using System.Collections;

In [104]:
string dbPath = "bosss_db_levelSets";
//string dbPath = "bosss_db_shockFinding";
var database=  OpenOrCreateDatabase(dbPath);

In [96]:
// Select sessions
List<ISessionInfo> sessions = new List<ISessionInfo>();
//sessions.AddRange(database.Sessions.Pick(6, 7, 8));
sessions.AddRange(database.Sessions.Pick(0));

In [97]:
// Select seeding setup
SeedingSetup seeding = SeedingSetup.av;
//SeedingSetup seeding = SeedingSetup.av3x3;
//SeedingSetup seeding = SeedingSetup.everywhere;

In [98]:
// Select the maximum number of iterations
int maxNumOfIterations = 100; 

In [100]:
// Create Direcotories
//string mainPath      = @"C:\tmp\ProductionRuns_Seeding_" + seeding.ToString() +  @"\";
//string mainPath        = @"C:\tmp\ProductionRuns_Seeding_" + seeding.ToString() +  @"_"+ maxNumOfIterations + @"\"; 
string mainPath = @"C:\tmp\XDGFields_" + seeding.ToString() +  @"_" + maxNumOfIterations.ToString() + @"\"; 

string[] sessionPathsRecon = ShockFindingExtensions.CreateDirectories(mainPath, @"LevelSetReconstruction", sessions);
ShockFindingExtensions.EmptyDirectories(mainPath, @"LevelSetReconstruction", sessions);

string[] sessionPathsInflec = ShockFindingExtensions.CreateDirectories(mainPath, @"InflectionPoints", sessions);


# Find the inflection points (takes time: about 400s for 1000 max Number of Iterations)

In [101]:
InflectionPointFinder finder = new InflectionPointFinder(sessionPathsInflec[0], session.Timesteps.Last());
    finder.FindPoints(seeding: seeding, patchRecoveryGradient: true, patchRecoveryHessian: true, eliminateNonConverged: true, maxNumOfIterations: maxNumOfIterations);
    //finder.Plot(plotDGFields: true, plotSeedingsPoints: true, plotInflectionsPoints: true, plotCurves: false, plotStartEndPairs: false);


Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at BoSSS.Solution.CompressibleFlowCommon.ShockFinding.ShockFindingExtensions.GetAVMeanValues(GridData gridData, SinglePhaseField avField) in C:\Users\jakob\Documents\Uni\Promotion\Programmieren\BoSSS\experimental\public\src\L3-solution\BoSSS.Solution.CompressibleFlowCommon\ShockFinding\ShockFindingExtensions.cs:line 209
   at BoSSS.Solution.CompressibleFlowCommon.ShockFinding.InflectionPointFinder.FindPoints(SeedingSetup seeding, Boolean patchRecoveryGradient, Boolean patchRecoveryHessian, Boolean eliminateNonConverged, Int32 maxNumOfIterations, Double eps) in C:\Users\jakob\Documents\Uni\Promotion\Programmieren\BoSSS\experimental\public\src\L3-solution\BoSSS.Solution.CompressibleFlowCommon\ShockFinding\InflectionPointFinder.cs:line 131
   at Submission#102.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecuti

In [28]:
finder.Results.SaveResults(finder.SessionPath);
finder.ResultsExtended.SaveResults(finder.SessionPath);

In [87]:
MultidimensionalArray results         = ShockFindingExtensions.LoadResults(sessionPathsInflec[0]);
MultidimensionalArray resultsExtended = ShockFindingExtensions.LoadResultsExtended(sessionPathsInflec[0]);

LevelSetReconstruction lsr = new LevelSetReconstruction(sessionPathsRecon[0], session, results, resultsExtended);
resultsExtended.SaveToTextFile("convergedCandidatePoints.txt");
clusteringZero.SaveToTextFile("clusteringZero.txt");
clusterZero.SaveToTextFile("clusterOne.txt");
clusteringTwo.SaveToTextFile("clusteringTwo.txt");
clusteringThree.SaveToTextFile("clusteringThree.txt");

In [88]:

// Clustering zero (density)
    MultidimensionalArray clusteringZero = lsr.CreateClustering_Density(3, new double[] {1.0, 2.7, 5.0});
    //lsr.SaveClusteringToTextFile(clusteringZero);
    MultidimensionalArray clusterZero = lsr.SelectCluster(clusteringZero, clusterToSelect: 1);
    
    // Clustering one (artificial viscosity)
    MultidimensionalArray clusteringOne = lsr.CreateClustering_AV(clusterZero, 2, new double[] {0.0, 0.0});
    lsr.SaveClusteringToTextFile(clusteringOne);
    //MultidimensionalArray clusterOne = lsr.SelectCluster(clusteringOne, clusterToSelect: 0);
    
    // Clustering two (eliminate boundary cells)
    MultidimensionalArray clusteringTwo = lsr.CreateClustering_Boundary(clusterZero);
    //lsr.SaveClusteringToTextFile(clusteringTwo);
    
    // Clustering three (eliminate cells close to the blunt body)
    MultidimensionalArray clusteringThree = lsr.CreateClustering_DeletePointsAtGeometry(clusteringTwo);
    //lsr.SaveClusteringToTextFile(clusteringThree);
    // Reconstruct level set field
    lsr.ReconstructLevelSet(patchRecovery: true, continuous: true);
    
    // Plot level set fields
    //lsr.PlotFields();


CreateClustering_Density: START
CreateClustering_Density: END
CreateClustering_AV: START


Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at BoSSS.Solution.CompressibleFlowCommon.ShockFinding.LevelSetReconstruction.CreateClustering_AV(MultidimensionalArray inputClustering, Int32 numOfClusters, Double[] initialMeans) in C:\Users\jakob\Documents\Uni\Promotion\Programmieren\BoSSS\experimental\public\src\L3-solution\BoSSS.Solution.CompressibleFlowCommon\ShockFinding\LevelSetReconstruction.cs:line 153
   at Submission#89.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [31]:
    
    // Save level set and conservative fields
    var exportDb = OpenOrCreateDatabase(lsr.SessionPath + @"bosss_db_levelSetReconstruction");

    DGField[] conservativeFields = new DGField[4];
    conservativeFields[0] = session.Timesteps.Last().Fields.Find("rho");
    conservativeFields[1] = session.Timesteps.Last().Fields.Find("m0");
    conservativeFields[2] = session.Timesteps.Last().Fields.Find("m1");
    conservativeFields[3] = session.Timesteps.Last().Fields.Find("rhoE");


Creating database 'C:\tmp\XDGFields_everywhere_100\IBMBowShock_p2_xCells40_yCells160_CFLFrac0.1_RK1_s0=1.0E-03_lambdaMax15\LevelSetReconstruction\bosss_db_levelSetReconstruction'.


In [32]:
    
    DGField[] levelSetFields = lsr.LevelSetFields.ToArray();


In [33]:

    DGField[] allFields = new DGField[levelSetFields.Length + conservativeFields.Length];


In [71]:
    levelSetFields.CopyTo(allFields, 0);


In [72]:
    conservativeFields.CopyTo(allFields, levelSetFields.Length);


In [74]:
allFields

index,value
0,<null>
1,levelSet_recon
2,levelSet_recon_prc
3,levelSet_recon_prc_cont
4,rho
5,m0
6,m1
7,rhoE


In [73]:
var texplot = new Tecplot(allFields.Pick(3).Basis.GridDat, 2);
texplot.PlotFields( "BS_"+ 0,0.0,allFields);
    

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at BoSSS.Solution.PlotDriver.<>c.<PlotFields>b__9_0(DGField x) in C:\Users\jakob\Documents\Uni\Promotion\Programmieren\BoSSS\experimental\public\src\L3-solution\BoSSS.Solution\PlotDriver.cs:line 1147
   at System.Linq.Utilities.<>c__DisplayClass2_0`3.<CombineSelectors>b__0(TSource x)
   at System.Linq.Enumerable.SelectArrayIterator`2.MoveNext()
   at BoSSS.Solution.Tecplot.Tecplot.OpenFile(String filename, IEnumerable`1 fieldNames) in C:\Users\jakob\Documents\Uni\Promotion\Programmieren\BoSSS\experimental\public\src\L3-solution\BoSSS.Solution.Tecplot\Tecplot.cs:line 102
   at BoSSS.Solution.PlotDriver.PlotFields(String fileNameBase, Double time, IEnumerable`1 fieldsToPlot) in C:\Users\jakob\Documents\Uni\Promotion\Programmieren\BoSSS\experimental\public\src\L3-solution\BoSSS.Solution\PlotDriver.cs:line 1129
   at BoSSS.Solution.PlotDriver.PlotFields(String fileNameBase, Double time, IEn

In [37]:
    
    exportDb.SaveGridAndTimestep(allFields, lsr.Session.ProjectName, lsr.Session.Name);



Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at BoSSS.Foundation.IO.IDatabaseInfoExtensions.SaveGridAndTimestep(IDatabaseInfo targetDb, DGField[] fields, String projectName, String sessionName) in C:\Users\jakob\Documents\Uni\Promotion\Programmieren\BoSSS\experimental\public\src\L4-application\BoSSSpad\IDatabaseInfoExtensions.cs:line 260
   at Submission#38.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [38]:
count


Error: (1,1): error CS0103: The name 'count' does not exist in the current context

In [39]:
Console.WriteLine("\n### LevelSetReconstruction.bws finished ###");


### LevelSetReconstruction.bws finished ###


In [42]:
// Test data
//MultidimensionalArray input = MultidimensionalArray.Create(3, 1, 5);
//input[0, 0, 0] = 0;
//input[0, 0, 1] = 0.1;
//input[0, 0, 2] = 0.2;
//input[0, 0, 3] = 0.3;
//input[0, 0, 4] = 0.4;

//input[1, 0, 0] = 1.0;
//input[1, 0, 1] = 1.1;
//input[1, 0, 2] = 1.2;
//input[1, 0, 3] = 1.3;
//input[1, 0, 4] = 1.4;

//input[2, 0, 0] = 2.0;
//input[2, 0, 1] = 2.1;
//input[2, 0, 2] = 2.2;
//input[2, 0, 3] = 2.3;
//input[2, 0, 4] = 2.4;

//MultidimensionalArray inputExtended = MultidimensionalArray.Create(3, 3);
//inputExtended[0, 0] = 1;
//inputExtended[0, 1] = 1;
//inputExtended[0, 2] = 0;

//inputExtended[1, 0] = 1;
//inputExtended[1, 1] = -1;
//inputExtended[1, 2] = 1;

//inputExtended[2, 0] = 1;
//inputExtended[2, 1] = 1;
//inputExtended[2, 2] = 2;